In [1]:
import pandas as pd

In [37]:
data_train=pd.read_csv("data/salary-train.csv")
data_test=pd.read_csv("data/salary-test-mini.csv")

In [38]:
data_train.FullDescription=map(str.lower,data_train.FullDescription)
data_test.FullDescription=map(str.lower,data_test.FullDescription)

In [39]:
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london ****k ****...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager// luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse / rgn nursing home for young...,Sutton Coldfield,NaN,20355


In [40]:
def replace(x):
     return x['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [41]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [42]:
data_train.FullDescription=replace(data_train)
data_test.FullDescription=replace(data_test)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
vector=TfidfVectorizer(min_df=5)
X_train1=vector.fit_transform(data_train.FullDescription)
X_test1=vector.transform(data_test.FullDescription)

In [44]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

In [45]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [47]:
print X_test_categ

  (0, 0)	1.0
  (0, 1040)	1.0
  (1, 2)	1.0
  (1, 997)	1.0


In [48]:
import scipy

In [49]:
print X_train1.shape
print X_train_categ.shape

(60000, 22861)
(60000, 1766)


In [52]:
X_train=scipy.sparse.hstack([X_train1,X_train_categ])
X_test=scipy.sparse.hstack([X_test1,X_test_categ])

In [53]:
print X_train.shape, X_test.shape

(60000, 24627) (2, 24627)


In [54]:
from sklearn.linear_model import Ridge

In [56]:
clf=Ridge(random_state=241)
clf.fit(X_train,data_train.SalaryNormalized)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [65]:
print 'salary: %2f, %2f:'%(clf.predict(X_test)[0],clf.predict(X_test)[1])

salary: 56555.615002, 37188.324426:
